# 📝 提示词工程与模板系统

## 🎯 学习目标

在本课程中，你将学习：
- 提示词工程的核心原理
- LangChain提示词模板系统
- 动态提示词构建技术
- 提示词优化策略
- 少样本学习和上下文学习
- 输出解析和结构化处理

---

In [ ]:
# 初始化学习环境
import sys
import os
sys.path.append('../utils')

from progress_tracker import ProgressTracker
import warnings
warnings.filterwarnings('ignore')

# 初始化进度追踪器
tracker = ProgressTracker()
lesson_id = "03_prompts_templates"
tracker.start_lesson(lesson_id, "提示词工程与模板系统")

print("🚀 欢迎来到提示词工程与模板系统课程！")
print("📊 正在初始化学习环境...")

# 显示学习进度
tracker.display_progress_summary()

## 1. 🎪 提示词工程基础

### 1.1 什么是提示词工程？

提示词工程是设计、优化和管理与大语言模型交互的输入文本的艺术与科学。

In [ ]:
# 环境配置检查
try:
    from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
    from langchain_core.prompts import FewShotPromptTemplate
    from langchain_core.example_selectors import SemanticSimilarityExampleSelector
    from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
    from langchain_openai import ChatOpenAI
    from dotenv import load_dotenv
    
    # 加载环境变量
    load_dotenv()
    
    print("✅ LangChain提示词模块加载成功")
    
    # 检查API密钥
    if os.getenv('OPENAI_API_KEY'):
        print("✅ OpenAI API密钥已配置")
        has_api_key = True
    else:
        print("⚠️ 未检测到OpenAI API密钥，将使用模拟模式")
        has_api_key = False
        
except ImportError as e:
    print(f"❌ 导入错误: {e}")
    print("💡 请运行: pip install langchain langchain-openai")
    has_api_key = False

### 1.2 基础提示词模板

In [ ]:
from langchain_core.prompts import PromptTemplate

# 创建简单的提示词模板
simple_template = PromptTemplate(
    input_variables=["topic", "audience"],
    template="请为{audience}写一篇关于{topic}的简短介绍。"
)

# 格式化提示词
prompt = simple_template.format(
    topic="人工智能",
    audience="初学者"
)

print("📝 基础提示词模板示例：")
print(f"模板: {simple_template.template}")
print(f"变量: {simple_template.input_variables}")
print(f"格式化结果: {prompt}")

# 记录完成状态
tracker.complete_exercise(lesson_id, "basic_prompt_template")

### 1.3 聊天提示词模板

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

# 创建聊天提示词模板
chat_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个专业的{role}，具有{experience}年的经验。请用专业但易懂的方式回答问题。"),
    ("human", "请解释：{question}")
])

# 格式化聊天提示词
messages = chat_template.format_messages(
    role="数据科学家",
    experience="5",
    question="什么是机器学习中的过拟合？"
)

print("💬 聊天提示词模板示例：")
for i, message in enumerate(messages):
    print(f"消息{i+1} ({message.__class__.__name__}): {message.content}")

# 记录完成状态
tracker.complete_exercise(lesson_id, "chat_prompt_template")

## 2. 🔧 高级提示词技术

### 2.1 少样本学习 (Few-Shot Learning)

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate

# 定义示例
examples = [
    {
        "input": "快乐",
        "output": "😊 今天阳光明媚，心情就像春天的花朵一样绽放。"
    },
    {
        "input": "悲伤",
        "output": "😢 雨滴敲打着窗户，就像心中的眼泪无声地流淌。"
    },
    {
        "input": "兴奋",
        "output": "🎉 就像火箭发射时的轰鸣，内心的激动难以抑制！"
    }
]

# 创建示例模板
example_template = PromptTemplate(
    input_variables=["input", "output"],
    template="情感: {input}\n描述: {output}"
)

# 创建少样本提示词模板
few_shot_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_template,
    prefix="请根据以下示例，为给定的情感创造富有诗意的描述：",
    suffix="情感: {emotion}\n描述:",
    input_variables=["emotion"]
)

# 生成提示词
prompt = few_shot_template.format(emotion="希望")

print("🎯 少样本学习提示词：")
print(prompt)
print("\n" + "="*50)

# 记录完成状态
tracker.complete_exercise(lesson_id, "few_shot_learning")

### 2.2 动态示例选择

In [ ]:
# 创建更多样化的示例数据库
emotion_examples = [
    {"emotion": "快乐", "context": "生日聚会", "description": "蛋糕上的蜡烛像星星一样闪烁，笑声填满了整个房间。"},
    {"emotion": "快乐", "context": "毕业典礼", "description": "学位帽飞向天空，梦想终于插上了翅膀。"},
    {"emotion": "悲伤", "context": "离别", "description": "火车站的汽笛声中，挥手告别的身影渐渐模糊。"},
    {"emotion": "悲伤", "context": "失败", "description": "努力如泡沫般破灭，留下的只有苦涩的回忆。"},
    {"emotion": "愤怒", "context": "不公", "description": "正义的火焰在胸中燃烧，愤怒如雷鸣般咆哮。"},
    {"emotion": "平静", "context": "海边", "description": "海浪轻抚着沙滩，心灵在这一刻找到了宁静的港湾。"},
]

# 创建简单的示例选择器（基于相似度）
def select_examples_by_emotion(target_emotion, examples, max_examples=2):
    """根据情感类型选择最相关的示例"""
    # 简单的关键词匹配选择
    relevant_examples = [ex for ex in examples if target_emotion.lower() in ex['emotion'].lower()]
    
    # 如果找不到直接匹配，选择前几个示例
    if not relevant_examples:
        relevant_examples = examples[:max_examples]
    
    return relevant_examples[:max_examples]

# 测试动态示例选择
target_emotion = "快乐"
selected_examples = select_examples_by_emotion(target_emotion, emotion_examples)

print(f"🎯 为情感 '{target_emotion}' 选择的示例：")
for i, example in enumerate(selected_examples, 1):
    print(f"\n示例 {i}:")
    print(f"情感: {example['emotion']}")
    print(f"场景: {example['context']}")
    print(f"描述: {example['description']}")

# 记录完成状态
tracker.complete_exercise(lesson_id, "dynamic_example_selection")

## 3. 📊 结构化输出与解析

### 3.1 输出解析器

In [ ]:
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_core.output_parsers import JsonOutputParser
from typing import List
import json

# 1. 字符串输出解析器
str_parser = StrOutputParser()

# 2. 逗号分隔列表解析器
list_parser = CommaSeparatedListOutputParser()

# 3. JSON输出解析器
json_parser = JsonOutputParser()

# 创建包含解析指令的提示词模板
list_template = PromptTemplate(
    template="列出5个关于{topic}的关键词。\n{format_instructions}",
    input_variables=["topic"],
    partial_variables={"format_instructions": list_parser.get_format_instructions()}
)

# 生成提示词
prompt = list_template.format(topic="机器学习")

print("📋 列表输出解析器示例：")
print(prompt)
print("\n" + "="*50)

# 模拟LLM输出并解析
mock_output = "监督学习, 无监督学习, 神经网络, 深度学习, 特征工程"
parsed_list = list_parser.parse(mock_output)

print("🔍 解析结果：")
print(f"原始输出: {mock_output}")
print(f"解析后列表: {parsed_list}")
print(f"数据类型: {type(parsed_list)}")

# 记录完成状态
tracker.complete_exercise(lesson_id, "output_parsers")

### 3.2 Pydantic结构化输出

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
from typing import List, Optional

# 定义数据模型
class BookReview(BaseModel):
    """书籍评论的结构化数据模型"""
    title: str = Field(description="书籍标题")
    author: str = Field(description="作者姓名")
    rating: int = Field(description="评分（1-5星）", ge=1, le=5)
    summary: str = Field(description="内容摘要")
    keywords: List[str] = Field(description="关键词列表")
    recommend: bool = Field(description="是否推荐")
    target_audience: Optional[str] = Field(description="目标读者群体", default=None)

# 创建Pydantic解析器
pydantic_parser = PydanticOutputParser(pydantic_object=BookReview)

# 创建结构化输出提示词模板
review_template = PromptTemplate(
    template="""请对以下书籍进行评论分析：

书籍：{book_info}

请提供结构化的评论信息：
{format_instructions}
""",
    input_variables=["book_info"],
    partial_variables={"format_instructions": pydantic_parser.get_format_instructions()}
)

# 生成提示词
prompt = review_template.format(
    book_info="《Python编程：从入门到实践》- Eric Matthes著"
)

print("📚 结构化输出提示词：")
print(prompt[:500] + "..." if len(prompt) > 500 else prompt)
print("\n" + "="*50)

# 模拟结构化输出
mock_structured_output = '''
{
    "title": "Python编程：从入门到实践",
    "author": "Eric Matthes",
    "rating": 4,
    "summary": "这是一本优秀的Python入门书籍，内容循序渐进，实例丰富，适合初学者学习。",
    "keywords": ["Python", "编程入门", "实战项目", "基础语法", "Web开发"],
    "recommend": true,
    "target_audience": "编程初学者和Python爱好者"
}
'''

try:
    # 解析结构化输出
    parsed_review = pydantic_parser.parse(mock_structured_output)
    
    print("✅ 解析成功！")
    print(f"书籍标题: {parsed_review.title}")
    print(f"作者: {parsed_review.author}")
    print(f"评分: {'⭐' * parsed_review.rating}")
    print(f"推荐: {'是' if parsed_review.recommend else '否'}")
    print(f"关键词: {', '.join(parsed_review.keywords)}")
    print(f"数据类型: {type(parsed_review)}")
    
except Exception as e:
    print(f"❌ 解析失败: {e}")

# 记录完成状态
tracker.complete_exercise(lesson_id, "pydantic_output")

## 4. 🚀 提示词优化策略

### 4.1 提示词版本管理

In [ ]:
from datetime import datetime
from typing import Dict, Any

class PromptVersionManager:
    """提示词版本管理器"""
    
    def __init__(self):
        self.versions = {}
        self.current_version = None
    
    def add_version(self, name: str, template: str, variables: List[str], 
                   description: str = "", metadata: Dict[str, Any] = None):
        """添加新版本的提示词"""
        version_info = {
            'template': template,
            'variables': variables,
            'description': description,
            'created_at': datetime.now(),
            'metadata': metadata or {}
        }
        
        self.versions[name] = version_info
        self.current_version = name
        return name
    
    def get_version(self, name: str) -> PromptTemplate:
        """获取指定版本的提示词模板"""
        if name not in self.versions:
            raise ValueError(f"版本 '{name}' 不存在")
        
        version_info = self.versions[name]
        return PromptTemplate(
            template=version_info['template'],
            input_variables=version_info['variables']
        )
    
    def list_versions(self):
        """列出所有版本"""
        print("📋 提示词版本列表：")
        for name, info in self.versions.items():
            current_marker = " (当前)" if name == self.current_version else ""
            print(f"\n🔖 {name}{current_marker}")
            print(f"   描述: {info['description']}")
            print(f"   创建时间: {info['created_at'].strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"   变量: {', '.join(info['variables'])}")

# 创建版本管理器实例
prompt_manager = PromptVersionManager()

# 添加不同版本的翻译提示词
prompt_manager.add_version(
    name="translate_v1",
    template="请将以下文本从{source_lang}翻译成{target_lang}：\n{text}",
    variables=["source_lang", "target_lang", "text"],
    description="基础翻译模板"
)

prompt_manager.add_version(
    name="translate_v2",
    template="""你是一个专业的翻译专家。请将以下{source_lang}文本翻译成{target_lang}，
保持原文的语调和风格：

原文：{text}

翻译：""",
    variables=["source_lang", "target_lang", "text"],
    description="改进的翻译模板，强调专业性和风格保持"
)

prompt_manager.add_version(
    name="translate_v3",
    template="""作为一名经验丰富的翻译专家，请执行以下翻译任务：

源语言：{source_lang}
目标语言：{target_lang}
专业领域：{domain}

原文：
{text}

翻译要求：
1. 保持原文的准确性和完整性
2. 适应目标语言的表达习惯
3. 保留专业术语的准确性

翻译：""",
    variables=["source_lang", "target_lang", "domain", "text"],
    description="专业翻译模板，增加领域专业性和详细要求"
)

# 显示版本列表
prompt_manager.list_versions()

# 记录完成状态
tracker.complete_exercise(lesson_id, "prompt_versioning")

### 4.2 A/B测试框架

In [ ]:
import random
from typing import List, Dict, Callable
import time

class PromptABTester:
    """提示词A/B测试框架"""
    
    def __init__(self):
        self.experiments = {}
        self.results = {}
    
    def create_experiment(self, experiment_name: str, 
                         prompt_variants: Dict[str, PromptTemplate],
                         evaluation_metric: Callable):
        """创建A/B测试实验"""
        self.experiments[experiment_name] = {
            'variants': prompt_variants,
            'evaluation_metric': evaluation_metric,
            'created_at': datetime.now()
        }
        
        # 初始化结果记录
        self.results[experiment_name] = {
            variant: {'responses': [], 'scores': [], 'avg_score': 0}
            for variant in prompt_variants.keys()
        }
    
    def run_test(self, experiment_name: str, test_data: List[Dict], 
                sample_ratio: float = 0.5):
        """运行A/B测试"""
        if experiment_name not in self.experiments:
            raise ValueError(f"实验 '{experiment_name}' 不存在")
        
        experiment = self.experiments[experiment_name]
        variants = list(experiment['variants'].keys())
        
        print(f"🧪 开始A/B测试: {experiment_name}")
        print(f"📊 测试变体: {', '.join(variants)}")
        print(f"📝 测试数据量: {len(test_data)}")
        
        for i, data in enumerate(test_data):
            # 随机分配到不同变体
            variant = random.choice(variants)
            
            # 获取对应的提示词模板
            template = experiment['variants'][variant]
            
            # 格式化提示词（这里模拟）
            formatted_prompt = template.format(**data)
            
            # 模拟LLM响应（实际使用时这里会调用真实的LLM）
            mock_response = f"[模拟响应] 使用{variant}处理: {data}"
            
            # 评估响应质量（这里使用随机分数模拟）
            score = random.uniform(0.6, 1.0)  # 模拟评分
            
            # 记录结果
            self.results[experiment_name][variant]['responses'].append(mock_response)
            self.results[experiment_name][variant]['scores'].append(score)
            
            if (i + 1) % 10 == 0:
                print(f"  处理进度: {i + 1}/{len(test_data)}")
        
        # 计算平均分数
        for variant in variants:
            scores = self.results[experiment_name][variant]['scores']
            if scores:
                avg_score = sum(scores) / len(scores)
                self.results[experiment_name][variant]['avg_score'] = avg_score
        
        print("✅ A/B测试完成")
    
    def get_results(self, experiment_name: str):
        """获取测试结果"""
        if experiment_name not in self.results:
            raise ValueError(f"实验 '{experiment_name}' 的结果不存在")
        
        results = self.results[experiment_name]
        
        print(f"📊 A/B测试结果: {experiment_name}")
        print("="*50)
        
        for variant, data in results.items():
            response_count = len(data['responses'])
            avg_score = data['avg_score']
            
            print(f"\n🔖 变体: {variant}")
            print(f"   响应数量: {response_count}")
            print(f"   平均评分: {avg_score:.3f}")
            print(f"   评分范围: {min(data['scores']):.3f} - {max(data['scores']):.3f}")
        
        # 找出最佳变体
        best_variant = max(results.keys(), key=lambda v: results[v]['avg_score'])
        best_score = results[best_variant]['avg_score']
        
        print(f"\n🏆 最佳变体: {best_variant} (评分: {best_score:.3f})")
        
        return results

# 创建A/B测试器实例
ab_tester = PromptABTester()

# 定义测试变体
summary_variants = {
    'variant_a': PromptTemplate(
        template="请总结以下文章：\n{article}",
        input_variables=["article"]
    ),
    'variant_b': PromptTemplate(
        template="请用3个要点总结以下文章的核心内容：\n{article}",
        input_variables=["article"]
    )
}

# 定义评估指标（这里使用简单的模拟）
def evaluate_summary_quality(response: str) -> float:
    """评估摘要质量的模拟函数"""
    # 实际使用中可能包含：
    # - 摘要长度适中性
    # - 关键信息覆盖度
    # - 语言流畅性
    # - 用户满意度评分
    return random.uniform(0.6, 1.0)

# 创建实验
ab_tester.create_experiment(
    experiment_name="summary_optimization",
    prompt_variants=summary_variants,
    evaluation_metric=evaluate_summary_quality
)

# 准备测试数据
test_articles = [
    {"article": "人工智能正在改变我们的生活方式..."},
    {"article": "机器学习算法在医疗诊断中的应用..."},
    {"article": "自然语言处理技术的最新发展..."},
    {"article": "深度学习在图像识别领域的突破..."},
    {"article": "量子计算对未来科技发展的影响..."}
] * 4  # 扩展到20个测试样本

# 运行A/B测试
ab_tester.run_test("summary_optimization", test_articles)

# 查看结果
ab_tester.get_results("summary_optimization")

# 记录完成状态
tracker.complete_exercise(lesson_id, "ab_testing")

## 5. 🎯 实战练习

### 练习1：创建多功能聊天机器人提示词

In [ ]:
# 练习：设计一个多功能聊天机器人的提示词系统

print("🎯 练习1：多功能聊天机器人提示词设计")
print("="*50)

# 任务：创建一个包含多种角色和功能的聊天机器人提示词模板
# 要求：
# 1. 支持不同的角色模式（助手、老师、朋友等）
# 2. 可以调整回答的详细程度
# 3. 支持不同的语言风格
# 4. 包含安全和伦理约束

chatbot_template = ChatPromptTemplate.from_messages([
    ("system", """
你是一个智能助手，具有以下特征：

🎭 角色模式：{role}
📊 详细程度：{detail_level}
🗣️ 语言风格：{style}
🌍 用户偏好：{user_preference}

请根据以上配置调整你的回答方式。始终保持友善、有帮助，并遵循以下原则：
- 提供准确和有用的信息
- 承认知识局限性
- 拒绝有害或不当请求
- 鼓励批判性思考
"""),
    ("human", "{user_message}")
])

# 测试不同配置
test_configs = [
    {
        "role": "专业导师",
        "detail_level": "详细解释",
        "style": "正式学术",
        "user_preference": "喜欢举例说明",
        "user_message": "请解释什么是机器学习？"
    },
    {
        "role": "友善伙伴",
        "detail_level": "简洁明了",
        "style": "轻松幽默",
        "user_preference": "喜欢类比和比喻",
        "user_message": "如何保持学习动力？"
    }
]

for i, config in enumerate(test_configs, 1):
    messages = chatbot_template.format_messages(**config)
    print(f"\n📝 配置{i}测试：")
    print(f"角色: {config['role']} | 风格: {config['style']}")
    print(f"用户问题: {config['user_message']}")
    print("\n系统提示词片段:")
    print(messages[0].content[:200] + "...")

print("\n✅ 练习1完成！你已经学会了如何设计灵活的聊天机器人提示词。")

# 记录练习完成
tracker.complete_exercise(lesson_id, "exercise_chatbot_prompts")

### 练习2：智能内容生成系统

In [ ]:
print("🎯 练习2：智能内容生成系统")
print("="*50)

# 任务：创建一个智能内容生成系统，能够：
# 1. 根据不同内容类型调整写作风格
# 2. 整合目标受众分析
# 3. 包含SEO优化要素
# 4. 支持多种输出格式

# 定义内容类型配置
content_configs = {
    "blog_post": {
        "structure": "引言-主体-结论",
        "tone": "信息性且引人入胜",
        "length": "800-1200字",
        "seo_focus": "关键词密度2-3%"
    },
    "product_description": {
        "structure": "特性-优势-行动召唤",
        "tone": "说服性和描述性",
        "length": "150-300字",
        "seo_focus": "产品关键词突出"
    },
    "social_media": {
        "structure": "钩子-内容-互动",
        "tone": "轻松活泼",
        "length": "50-280字",
        "seo_focus": "话题标签优化"
    }
}

# 创建智能内容生成模板
content_generation_template = PromptTemplate(
    template="""
📝 智能内容生成任务

内容类型：{content_type}
主题：{topic}
目标受众：{target_audience}
关键词：{keywords}
语调：{tone}

结构要求：{structure}
长度要求：{length}
SEO要求：{seo_focus}

请根据以上要求创作内容。确保：
1. 内容与主题高度相关
2. 语调适合目标受众
3. 自然融入关键词
4. 遵循指定的结构和长度
5. 内容原创且有价值

开始创作：
""",
    input_variables=["content_type", "topic", "target_audience", "keywords", 
                    "tone", "structure", "length", "seo_focus"]
)

# 测试内容生成
test_content_request = {
    "content_type": "博客文章",
    "topic": "Python自动化脚本入门",
    "target_audience": "编程初学者",
    "keywords": "Python自动化, 脚本编程, 工作效率",
    **content_configs["blog_post"]
}

# 生成提示词
content_prompt = content_generation_template.format(**test_content_request)

print("📋 生成的内容创作提示词：")
print(content_prompt)

print("\n🎨 提示词特点分析：")
print("✅ 结构化参数配置")
print("✅ 多维度内容要求")
print("✅ SEO优化指导")
print("✅ 质量控制约束")

# 记录练习完成
tracker.complete_exercise(lesson_id, "exercise_content_generation")

## 6. 📊 学习总结与进度回顾

In [ ]:
# 完成课程并生成学习报告
completion_time = tracker.complete_lesson(lesson_id)

print("🎓 提示词工程与模板系统 - 课程完成！")
print("="*60)

print("\n📚 本课程核心知识点：")
key_concepts = [
    "提示词模板的创建和使用",
    "聊天提示词的消息结构",
    "少样本学习技术",
    "动态示例选择策略",
    "结构化输出解析",
    "Pydantic数据模型集成",
    "提示词版本管理",
    "A/B测试框架设计",
    "多功能提示词系统",
    "智能内容生成应用"
]

for i, concept in enumerate(key_concepts, 1):
    print(f"   {i:2d}. {concept}")

print("\n🛠️ 实践技能收获：")
skills = [
    "掌握LangChain提示词模板API",
    "设计灵活的提示词结构",
    "实现结构化数据输出",
    "构建提示词优化工作流",
    "开发智能内容生成系统"
]

for i, skill in enumerate(skills, 1):
    print(f"   ⚡ {skill}")

print("\n🎯 下一步学习建议：")
next_steps = [
    "学习Chain的组合和链接（04_chains_introduction.ipynb）",
    "探索Agent智能代理系统",
    "深入记忆系统和上下文管理",
    "实践更复杂的提示词工程项目"
]

for i, step in enumerate(next_steps, 1):
    print(f"   📈 {step}")

# 显示整体学习进度
print("\n" + "="*60)
tracker.display_progress_summary()

print("\n🎉 恭喜完成提示词工程与模板系统课程！")
print("💪 你已经掌握了LangChain中最重要的提示词技术！")
print("🚀 继续学习Chain和Agent，探索更强大的AI应用！")